In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import seaborn as sns

In [3]:
!pip3 install gensim
!pip3 install pyLDAvis
# !conda init bash


In [4]:
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
import pyLDAvis
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample

In [5]:
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)
from src.loader import SlackDataLoader
from src.preprocessing import Preprocessor
slack_data = SlackDataLoader('../anonymized/')
data = slack_data.slack_parser('../anonymized/*/')

[nltk_data] Downloading package wordnet to /home/basilel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/basilel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
preprocess = Preprocessor(data)

In [7]:
data.head()

,msg_type,msg_content,sender_name,msg_sent_time,msg_dist_type,time_thread_start,reply_count,reply_users_count,reply_users,tm_thread_end,channel
0,message,<!here> we're on gmeet...if you're online join us,Judith Bolton,1663139588.116659,broadcast,0,0,0,0,0,
1,message,<@U03UH397319> <https://anderfernandez.com/en/...,Judith Bolton,1663142046.799109,user,0,0,0,0,0,
2,message,when running your API try `python3 -m uvicorn ...,Judith Bolton,1663142094.022539,text,0,0,0,0,0,
3,message,Well received with thanks <@U03UJGP0C68>,Brenda Hernandez,1663142486.253059,text,0,0,0,0,0,
4,message,Good afternoon everyone. I am sorry for having...,Brian Odom,1663151073.400859,text,1663151073.400859,2,2,"U03UH397319,U03UJKJGRAQ",1663154349.239879,


In [8]:
data.shape

(17748, 11)

In [9]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [10]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

data = preprocess.clean_text(org_col='msg_content',new_col='clean_msg_content')
data = preprocess.stem('clean_msg_content')

In [11]:
data['msg_content'][2]

'when running your API try `python3 -m uvicorn main:app --reload` or `python -m uvicorn main:app --reload`'

In [12]:
data['clean_msg_content'][2]

'run api tri `python3 -m uvicorn main:app --reload` `python -m uvicorn main:app --reload`'

In [13]:
data.to_csv('../clean_messages.csv')

In [14]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/basilel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/basilel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# Used stemmer instead of lemmatizer
cleanMessageList = preprocess.filterMessageList(data['clean_msg_content'])

In [16]:
cleanMessageList[:5]

[['<here>', 'gmeetif', 'onlin', 'join'],
 ['<uuh>', '<httpsanderfernandezcomenbloghow-to-create-api-python>'],
 ['run',
  'api',
  'tri',
  '`python',
  '-m',
  'uvicorn',
  'mainapp',
  '--reload`',
  '`python',
  '-m',
  'uvicorn',
  'mainapp',
  '--reload`'],
 ['well', 'receiv', 'thank', '<uujgpc>'],
 ['good',
  'afternoon',
  'everyone',
  'sorri',
  'miss',
  'meet',
  'earlier',
  'feel',
  'well',
  'today']]

In [17]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=50,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [18]:
"""Create model objects"""
kagDict   = makeDict(cleanMessageList)
kagCorpus = makeCorpus(cleanMessageList, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

"""Save model objects"""
SaveLoad.save(kagLda,'kaggleLDAmodel')
corpora.MmCorpus.serialize('kaggleCorpus.mm', kagCorpus)
kagDict.save('kaggleDictionary.dict')

In [19]:
kagLda = SaveLoad.load('kaggleLDAmodel')
kagDict = corpora.Dictionary.load('kaggleDictionary.dict')
kagCorpus = corpora.MmCorpus('kaggleCorpus.mm')

In [20]:
!pip3 install pyLDAvis

In [21]:
# import pyLDAvis
import pyLDAvis.gensim_models

In [22]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim_models.prepare(kagLda, kagCorpus, kagDict)

In [23]:
kagLda.print_topics()

[(6,
  '0.091*"rolling_on_the_floor_laughing" + 0.082*"<uugjpc>" + 0.044*"<uvamtfa>" + 0.044*"<uugqv>" + 0.038*"happi" + 0.031*"notebook" + 0.023*"jupyt" + 0.022*"said" + 0.021*"<uuunmrx>" + 0.017*"birthday"'),
 (12,
  '0.081*"add" + 0.073*"git" + 0.053*"dvc" + 0.045*"remot" + 0.034*"function" + 0.031*"still" + 0.030*"push" + 0.018*"commit" + 0.016*"tag" + 0.013*"<httpgacademyorg|gacademyorg>"'),
 (35,
  '0.119*"<uuvhcvkb>" + 0.075*"done" + 0.043*"someth" + 0.040*"like" + 0.020*"part" + 0.020*"import" + 0.014*"time" + 0.013*"almost" + 0.013*"fun" + 0.012*"think"'),
 (19,
  '0.114*"great" + 0.090*"ok" + 0.062*"step" + 0.048*"learn" + 0.032*"<uuhagtr>" + 0.019*"work" + 0.018*"repositori" + 0.018*"fast" + 0.016*"mlflow" + 0.015*"order"'),
 (0,
  '0.032*"fix" + 0.031*"world" + 0.024*"=" + 0.023*"cat" + 0.023*"port" + 0.023*"<uufwpnce>" + 0.023*"messag" + 0.020*"`" + 0.019*"cta" + 0.019*"gt"'),
 (46,
  '0.043*"idea" + 0.037*"provid" + 0.037*"ubuntu" + 0.036*"email" + 0.031*"resourc" + 0.027

In [24]:
ldaViz

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
41     0.391792  0.088040       1        1  6.558648
31     0.112109 -0.006941       2        1  3.708432
27     0.256497  0.005952       3        1  3.353982
30     0.034243 -0.075088       4        1  3.308796
1      0.020242  0.042498       5        1  2.566243
7      0.277108  0.002571       6        1  2.423154
11    -0.049768  0.157616       7        1  2.385733
28     0.071532 -0.027535       8        1  2.378582
6     -0.055400 -0.070490       9        1  2.347982
34    -0.036400 -0.027117      10        1  2.236008
23    -0.066248  0.247207      11        1  2.223655
42     0.010381 -0.006007      12        1  2.213770
43    -0.014578  0.009359      13        1  2.193617
24    -0.066752  0.191884      14        1  2.130550
5     -0.027211  0.143577      15        1  2.114222
15    -0.042829  0.111195      16        1  2.044123
2      0.113451  0.053346      17        1  2.034921
44    -0.029179  0.001524      18        1  2.027789
47     0.013916 -0.051538      19        1  1.992277
39     0.007550 -0.005387      20        1  1.990885
29    -0.051464  0.013171      21        1  1.972221
33     0.080375 -0.037802      22        1  1.906007
45     0.022430 -0.024596      23        1  1.898041
17    -0.075036  0.116662      24        1  1.886747
36    -0.019953 -0.043620      25        1  1.845076
13    -0.053442  0.047907      26        1  1.836105
37    -0.023725 -0.057418      27        1  1.768151
26    -0.024484 -0.073933      28        1  1.754008
19    -0.040759  0.030463      29        1  1.753752
32    -0.004785 -0.071242      30        1  1.750076
20    -0.054287 -0.023535      31        1  1.704907
10    -0.004961 -0.011740      32        1  1.687190
46    -0.021589 -0.069709      33        1  1.633979
21    -0.040512  0.022432      34        1  1.625846
3     -0.026293 -0.047355      35        1  1.619085
18    -0.019800 -0.078225      36        1  1.596503
16    -0.024905 -0.010412      37        1  1.586763
22    -0.060230  0.083488      38        1  1.558023
35    -0.039340 -0.087580      39        1  1.521417
14    -0.071582 -0.033678      40        1  1.444125
9     -0.032641 -0.011440      41        1  1.442614
25    -0.042365 -0.072422      42        1  1.437468
38    -0.037519 -0.078543      43        1  1.428856
49    -0.033075 -0.060566      44        1  1.415436
12    -0.073480  0.084570      45        1  1.390526
8     -0.032586 -0.064128      46        1  1.368062
0     -0.042228 -0.038060      47        1  1.346558
40    -0.011101 -0.082698      48        1  1.336580
4     -0.028470 -0.024001      49        1  1.139025
48    -0.032649 -0.080656      50        1  1.113484, topic_info=         Term        Freq       Total Category  logprob  loglift
16      thank  712.000000  712.000000  Default  30.0000  30.0000
22       good  476.000000  476.000000  Default  29.0000  29.0000
177        ye  360.000000  360.000000  Default  28.0000  28.0000
12        tri  581.000000  581.000000  Default  27.0000  27.0000
57       work  942.000000  942.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
454   project   15.888327  132.865185  Topic50  -4.1604   2.3739
927   trainee    7.751853   22.156561  Topic50  -4.8781   3.4475
111     video   12.576155  104.997650  Topic50  -4.3942   2.3755
144      imag    8.780674  116.410695  Topic50  -4.7535   1.9131
1555   includ    8.053789   49.196445  Topic50  -4.8399   2.6880

[2480 rows x 6 columns], token_table=      Topic      Freq                                               Term
term                                                                    
671      49  0.681500  "homejdsdocumentsdvcessyvenvlibpythonsite-pack...
548      26  0.744304  "homejdsdocumentsdvcessyvenvlibpythonsite-pack...
550      26  0.874599  "homejdsdocumentsdvcessyvenvlibpythonsite-pack...
552      26  0.744

In [25]:
def translateLdaIdx(myLdaModel, myLdaViz):
    """Translate lda model topics to match the topics in pyLDAvis visualization"""
    ldaVizIdx = myLdaViz[0].index
    return list(ldaVizIdx)

In [26]:
newIdx = translateLdaIdx(kagLda,ldaViz)

In [27]:
def createDenseMat(myLdaModel,myCorpus,newIdx):
    """Transform corpus to dataframe with topics matching lda visualization"""
    numTopics = myLdaModel.num_topics
    myDense = corpus2dense(myLdaModel[myCorpus],numTopics)
    myDf = pd.DataFrame(myDense)
    mySortedDf = myDf.transpose()
    mySortedDf = myDf.transpose()[newIdx]
    mySortedDf.columns = ['topic' + str(i + 1) for i in range(numTopics)]
    return mySortedDf

In [28]:
kagDf = createDenseMat(kagLda,kagCorpus,newIdx)

In [29]:
import numpy as np

def sortByTopicToIdx(cleanedTweetList, mySortedDf, myTopic, myTopicThresh=0.1):
    """Returns an index of tweets surpassing a topic value threshold"""
    myCleanArray = np.array(cleanedTweetList, dtype=object)
    srtIdx = list(mySortedDf[mySortedDf[myTopic] > myTopicThresh].index)
    return srtIdx

# Example usage
sortedIdx = sortByTopicToIdx(cleanMessageList, kagDf, 'topic2', myTopicThresh=0.1)


def sortTweetsByIdx(cleanedTweetList,srtIdx):
    """Returns sorted tweets as a list based on a defined sort index"""
    myCleanArray = np.array(cleanedTweetList, dtype=object)
    srtTweets = list(myCleanArray[srtIdx])
    return srtTweets

In [30]:
!pip3 install numpy


In [31]:
sortedIdx = sortByTopicToIdx(cleanMessageList,kagDf,'topic2',myTopicThresh=0.1)

In [34]:
# Word cloud for topic 
def makeWordCloud(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Create word cloud of tweets passing a given threshold for a given topic"""
    sortedIdx = sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1)
    mySortedTweets = sortTweetsByIdx(cleanedTweetList,sortedIdx)
    filteredWords = ' '.join([' '.join(string) for string in mySortedTweets])
    myTopicCloud = WordCloud(max_font_size=100,scale=8).generate(filteredWords)
    fig = plt.figure(figsize=(10,10), dpi=1600)
    plt.imshow(myTopicCloud)
    plt.axis("off")
    plt.show()

In [35]:
# Example usage
makeWordCloud(cleanMessageList,kagDf,'topic2',myTopicThresh=0.2)

/home/basilel/anaconda3/lib/python3.11/site-packages/wordcloud/wordcloud.py:106: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  self.colormap = plt.cm.get_cmap(colormap)
